# Routing Topology: OSPF using FRRouting

This notebook is an example of how to create a FABRIC routing experiment topology comprising nodes at three different sites. Each site has a local layer 2 (Ethernet) network connecting a set of local nodes and one gateway router. The three gateway routers connect to each other and use the [FRRouting](https://frrouting.org/) protocol suite to deploy [OSPF](https://en.wikipedia.org/wiki/Open_Shortest_Path_First) dameons to propagate route updates across the topology.

You might be familiar with the [Quagga](https://www.quagga.net/) router suite.  FRRouting is based on Quagga but has a more active upstream community including many large companies working on cloud networking.


## Step 1:  Configure the Environment


In [ ]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']=<INSERT_YOUR_FABRIC_USERNAME>
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_bastion_key'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Step 2: Import the FABLlib Library


In [ ]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

## Step 3 (Optional): Query for Available Tesbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

## Step 4: Create the Experiment Slice

The following creates private layer 2 networks on three sites including a OSPF gateway routers that propogate routes acrross the topology. 


In [ ]:
slice_name = 'OSPF_Routing_Topology'

[site1,site2,site3] = fablib.get_random_sites(count=3)
print(f"Sites: {site1},{site2},{site3}")

router_base_name='router'
router_link_base_name='router_link'

node_base_name='node'
local_network_base_name='net_local'

site_node_count=2


In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    #Create Router1
    router1_name = f"{router_base_name}1"
    router1 = slice.add_node(name=router1_name, site=site1)
    router1_iface1 = router1.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
    router1_iface2 = router1.add_component(model='NIC_Basic', name='nic2').get_interfaces()[0]
    router1_local_iface = router1.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]


    #Create Router2
    router2_name = f"{router_base_name}2"
    router2 = slice.add_node(name=router2_name, site=site2)
    router2_iface1 = router2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
    router2_iface2 = router2.add_component(model='NIC_Basic', name='nic2').get_interfaces()[0]
    router2_local_iface = router2.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]


    #Create Router3
    router3_name = f"{router_base_name}3"
    router3 = slice.add_node(name=router3_name, site=site3)
    router3_iface1 = router3.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
    router3_iface2 = router3.add_component(model='NIC_Basic', name='nic2').get_interfaces()[0]
    router3_local_iface = router3.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]


    #Create Router Links
    route_link1_name = f'{router_link_base_name}1'
    router_link1 = slice.add_l2network(name=route_link1_name, interfaces=[router1_iface1, router2_iface2])
    
    route_link2_name = f'{router_link_base_name}2'
    router_link2 = slice.add_l2network(name=route_link2_name, interfaces=[router2_iface1, router3_iface2])
    
    route_link3_name = f'{router_link_base_name}3'
    router_link3 = slice.add_l2network(name=route_link3_name, interfaces=[router3_iface1, router1_iface2])

    #Create Site1 Nodes
    site1_local_ifaces = [ router1_local_iface ]
    for i in range(site_node_count):
        node = slice.add_node(name=f'{node_base_name}_{site1}_{i+1}', site=site1)
        site1_local_ifaces.append(node.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0])
        
    #Create Site2 Nodes
    site2_local_ifaces = [ router2_local_iface ]
    for i in range(site_node_count):
        node = slice.add_node(name=f'{node_base_name}_{site2}_{i+1}', site=site2)
        site2_local_ifaces.append(node.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0])

    #Create Site1 Nodes
    site3_local_ifaces = [ router3_local_iface ]
    for i in range(site_node_count):
        node = slice.add_node(name=f'{node_base_name}_{site3}_{i+1}', site=site3)
        site3_local_ifaces.append(node.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0])
        
    #Create Site Local Networks
    site1_local_net_name = f'{local_network_base_name}1'
    site2_local_net_name = f'{local_network_base_name}2'
    site3_local_net_name = f'{local_network_base_name}3'
    
    site1_local_net = slice.add_l2network(name=site1_local_net_name, interfaces=site1_local_ifaces)
    site2_local_net = slice.add_l2network(name=site2_local_net_name, interfaces=site2_local_ifaces)
    site2_local_net = slice.add_l2network(name=site3_local_net_name, interfaces=site3_local_ifaces)

    #Submit Slice Request
    slice_id = slice.submit()

except Exception as e:
    print(f"Slice Fail: {e}")

## Step 5: Observe the Slice's Attributes

### Print the slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

### Print the Node List

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

### Print the Node Details

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

### Print the Node SSH Commands

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node.get_name()}: {node.get_ssh_command()}")
except Exception as e:
    print(f"Exception: {e}")

### Print the Interfaces

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

## Step 6: Configure IP Addresses

This experiment includes six different networks, each of which need to have a distinct subnet. The following cells show how to pick IPv4 subnets for each network and assign addresses from those subnets to the appropriate interfaces on the nodes and routers.


### Pick Subnets

Pick the subnet for the routing links. Each routing link connects a pair of routers. Although these links always have exactly two interfaces, we choose a /24 subnet for easy readability. For each link we create a subnet and a list of available IPs for that subnet. These will be used later to configure the router interfaces connected to these links. 

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    # Route Link Subnets
    route_link1_subnet = IPv4Network("192.168.101.0/24")
    route_link1_available_ips = list(route_link1_subnet)[1:]
    route_link1_addr1 = route_link1_available_ips.pop(0)
    route_link1_addr2 = route_link1_available_ips.pop(0)
    
    route_link2_subnet = IPv4Network("192.168.102.0/24")
    route_link2_available_ips = list(route_link2_subnet)[1:]
    route_link2_addr1 = route_link2_available_ips.pop(0)
    route_link2_addr2 = route_link2_available_ips.pop(0)
    
    route_link3_subnet = IPv4Network("192.168.103.0/24")
    route_link3_available_ips = list(route_link3_subnet)[1:]
    route_link3_addr1 = route_link3_available_ips.pop(0)
    route_link3_addr2 = route_link3_available_ips.pop(0)
    
    print(f"Router Link 1: subnet: {route_link1_subnet}, addr1: {route_link1_addr1}, addr2: {route_link1_addr2}")
    print(f"Router Link 2: subnet: {route_link2_subnet}, addr1: {route_link2_addr1}, addr2: {route_link2_addr2}")
    print(f"Router Link 3: subnet: {route_link3_subnet}, addr1: {route_link3_addr1}, addr2: {route_link3_addr2}")

except Exception as e:
    print(f"Exception: {e}")
   

Pick the subnet for the local networks.  The /24 subnets can support up to 254 locally connected nodes plus the gateway. 


In [ ]:
try:
    # Local Subnets
    net_local1_subnet = IPv4Network("192.168.1.0/24")
    net_local1_available_ips = list(net_local1_subnet)[1:]
    net_local1_gateway = net_local1_available_ips.pop(0)
        
    net_local2_subnet = IPv4Network("192.168.2.0/24")
    net_local2_available_ips = list(net_local2_subnet)[1:]
    net_local2_gateway = net_local2_available_ips.pop(0)


    net_local3_subnet = IPv4Network("192.168.3.0/24")
    net_local3_available_ips = list(net_local3_subnet)[1:]
    net_local3_gateway = net_local3_available_ips.pop(0)

    print(f"Site1: subnet: {net_local1_subnet}, gateway: {net_local1_gateway}")
    print(f"Site2: subnet: {net_local2_subnet}, gateway: {net_local2_gateway}")
    print(f"Site3: subnet: {net_local3_subnet}, gateway: {net_local3_gateway}")

except Exception as e:
    print(f"Exception: {e}")

### Configure Router IPs

Add the IPs to the corrisponing interfaces.

In [ ]:
try:
    # Config Router1 IPs
    router1 = slice.get_node(name=router1_name)  
    router1_iface1 = router1.get_interface(network_name=route_link1_name)  
    router1_iface2 = router1.get_interface(network_name=route_link3_name)  
    router1_local_iface = router1.get_interface(network_name=site1_local_net_name)     
    router1_iface1.ip_addr_add(addr=route_link1_addr1, subnet=route_link1_subnet)
    router1_iface2.ip_addr_add(addr=route_link3_addr2, subnet=route_link3_subnet)
    router1_local_iface.ip_addr_add(addr=net_local1_gateway, subnet=net_local1_subnet)
    
    # Config Router2 IPs
    router2 = slice.get_node(name=router2_name)  
    router2_iface1 = router2.get_interface(network_name=route_link2_name)  
    router2_iface2 = router2.get_interface(network_name=route_link1_name)  
    router2_local_iface = router2.get_interface(network_name=site2_local_net_name)     
    router2_iface1.ip_addr_add(addr=route_link2_addr1, subnet=route_link2_subnet)
    router2_iface2.ip_addr_add(addr=route_link1_addr2, subnet=route_link1_subnet)
    router2_local_iface.ip_addr_add(addr=net_local2_gateway, subnet=net_local2_subnet)
    
    # Config Router3 IPs
    router3 = slice.get_node(name=router3_name) 
    router3_iface1 = router3.get_interface(network_name=route_link3_name)  
    router3_iface2 = router3.get_interface(network_name=route_link2_name)  
    router3_local_iface = router3.get_interface(network_name=site3_local_net_name)     
    router3_iface1.ip_addr_add(addr=route_link3_addr1, subnet=route_link3_subnet)
    router3_iface2.ip_addr_add(addr=route_link2_addr2, subnet=route_link2_subnet)
    router3_local_iface.ip_addr_add(addr=net_local3_gateway, subnet=net_local3_subnet)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()
    

(Optional) Print the network interface configuration for each router.

In [ ]:
try:
    for router in [router1, router2, router3]:
        print(f'{router.get_name()}:')
        stdout, stderr = router.execute(f'ip addr list')
        print (stdout)
        print (stderr)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

### Configure Local Node IPs

Configure the local nodes with addresses from the local subnet available address list. Add routes to the other local subnets via the local gateway.  

Add the router link subnets if you want to access the routers (i.e. if you want to `ping` the routers or find paths with `tracepath`)

Collect a list of local dataplane IPs to target for testing

In [ ]:
local_dataplane_ips = {}

In [ ]:
try:    
    #Create Site1 Nodes
    for i in range(site_node_count):
        name=f'{node_base_name}_{site1}_{i+1}'
        node = slice.get_node(name=name)
        node_addr = net_local1_available_ips.pop(0)
        node_iface = node.get_interface(network_name=site1_local_net_name)  
        node_iface.ip_addr_add(addr=node_addr, subnet=net_local1_subnet)
        
        #Add routes to other local subnets
        node.ip_route_add(subnet=net_local2_subnet, gateway=net_local1_gateway)
        node.ip_route_add(subnet=net_local3_subnet, gateway=net_local1_gateway)
        
        #Add routes to router subnets (used for tracepath and pinging router interfaces)
        node.ip_route_add(subnet=route_link1_subnet, gateway=net_local1_gateway)
        node.ip_route_add(subnet=route_link2_subnet, gateway=net_local1_gateway)
        node.ip_route_add(subnet=route_link3_subnet, gateway=net_local1_gateway)
        
        #Collect dataplane IP for testing
        local_dataplane_ips[name] = node_addr
        
        print(f"Node {name} dataplane IP: {node_addr}")    
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
        print (stdout)
except Exception as e:
    print(f"Exception: {e}")
    

In [ ]:
try:
    #Create Site2 Nodes
    for i in range(site_node_count):
        name=f'{node_base_name}_{site2}_{i+1}'
        node = slice.get_node(name=name)
        node_addr = net_local2_available_ips.pop(0)
        node_iface = node.get_interface(network_name=site2_local_net_name)  
        node_iface.ip_addr_add(addr=node_addr, subnet=net_local2_subnet)
        
        #Add routes to other local subnets
        node.ip_route_add(subnet=net_local1_subnet, gateway=net_local2_gateway)
        node.ip_route_add(subnet=net_local3_subnet, gateway=net_local2_gateway)
        
         #Add routes to router subnets (used for tracepath and pinging router interfaces)
        node.ip_route_add(subnet=route_link1_subnet, gateway=net_local2_gateway)
        node.ip_route_add(subnet=route_link2_subnet, gateway=net_local2_gateway)
        node.ip_route_add(subnet=route_link3_subnet, gateway=net_local2_gateway)
        
        #Collect dataplane IP for testing
        local_dataplane_ips[name] = node_addr

        print(f"Node {name} dataplane IP: {node_addr}")
        stdout, stderr = node.execute(f'ip addr list')
        print (stdout)
        
        stdout, stderr = node2.execute(f'ip addr show {node_iface.get_os_interface()}')
        print (stdout)
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    #Create Site3 Nodes
    for i in range(site_node_count):
        name=f'{node_base_name}_{site3}_{i+1}'
        node = slice.get_node(name=name)
        node_addr = net_local3_available_ips.pop(0)
        node_iface = node.get_interface(network_name=site3_local_net_name)  
        node_iface.ip_addr_add(addr=node_addr, subnet=net_local3_subnet)
        
        #Add routes to other local subnets
        node.ip_route_add(subnet=net_local1_subnet, gateway=net_local3_gateway)
        node.ip_route_add(subnet=net_local2_subnet, gateway=net_local3_gateway)
        
        #Add routes to router subnets (used for tracepath and pinging router interfaces)
        node.ip_route_add(subnet=route_link1_subnet, gateway=net_local3_gateway)
        node.ip_route_add(subnet=route_link2_subnet, gateway=net_local3_gateway)
        node.ip_route_add(subnet=route_link3_subnet, gateway=net_local3_gateway)
        
        #Collect dataplane IP for testing
        local_dataplane_ips[name] = node_addr
        
        print(f"Node {name} dataplane IP: {node_addr}")
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
        print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

##  Step 7: Configure FRRouting on each router. 

This complex configuration is handled through a bash script `frr_config.sh` that resides in the folder containing this notebook. The script is executed by, first, uploading the script with the `node.upload_file()` FABLib method.  Then the script is executed using the `node.execute()` FABLib method.  Note that the script passes the OS interfaces names and configured IPs as arguments from the notebook to the script.  

These scripts take a while to run. You may wish to use a separate terminal window to ssh to the routers and tail the log file to watch the progess with: `tail -F frr_config.log`.

In [ ]:
try:  
    # Config Router1
    print('Config Router1')
    router1.upload_file('./frr_config.sh','frr_config.sh')
    stdout, stderr = router1.execute(f'chmod +x frr_config.sh && sudo ./frr_config.sh {router1_iface1.get_os_interface()} {route_link1_addr1} {router1_iface2.get_os_interface()} {route_link3_addr2} {router1_local_iface.get_os_interface()} {net_local1_gateway} 192.168.0.0')
    print (stdout)
    print (stderr)

    # Config Router2
    print('Config Router2')
    router2.upload_file('./frr_config.sh','frr_config.sh')    
    stdout, stderr = router2.execute(f'chmod +x frr_config.sh && sudo ./frr_config.sh {router2_iface1.get_os_interface()} {route_link2_addr1} {router2_iface2.get_os_interface()} {route_link1_addr2} {router2_local_iface.get_os_interface()} {net_local2_gateway} 192.168.0.0')
    print (stdout)
    print (stderr)
    
    # Config Router3
    print('Config Router3')
    router3.upload_file('./frr_config.sh','frr_config.sh')
    stdout, stderr = router3.execute(f'chmod +x frr_config.sh && sudo ./frr_config.sh {router3_iface1.get_os_interface()} {route_link3_addr1} {router3_iface2.get_os_interface()} {route_link2_addr2} {router3_local_iface.get_os_interface()} {net_local3_gateway} 192.168.0.0')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()            
     

## Step 8: Run the Experiemnt

We will just test `ping` RTT and look at `tracepath`. Your experiment should be more interesting!

Notice that if you run this quickly and repeatedly run this test against a specific target, you may see changes to the tracepath.  Initially the ping may even fail.  Why do you think this is happening?


In [ ]:
try:
    source_node_name =  f'{node_base_name}_{site1}_1'
   
    source_node = slice.get_node(name=source_node_name)
    for node_name,target_ip in local_dataplane_ips.items():
        print(f"Testing target node: {node_name}, target IP: {target_ip}")
    
        stdout, stderr = node.execute(f'ping -c 5 {target_ip}')
        print (stdout, stderr)

        stdout, stderr = node.execute(f'tracepath {target_ip}')
        print (stdout, stderr)
    
except Exception as e:
    print(f"Exception: {e}")

## Step 9: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    #slice.delete()
except Exception as e:
    print(f"Exception: {e}")